# RACE数据集，英语阅读理解多项选择QA

1. 使用RoBERTa，基于multichoice头，完成模型微调（不同类型的QA任务是不同的）
2. 尝试更换为MLM任务进行模型微调
3. 了解DataCollator
4. 自定义Pipeline


# 加载数据集
使用 datasets 库加载 imdb 数据集。这个库会自动下载并缓存数据。
也可以下载到本地，这样可以防止网络问题导致代码执行失败（尽管已经下载到缓存）


In [1]:
from datasets import load_dataset

dataset_name = "/data/Dataset/LLM-dataset/race"
raw_dataset = load_dataset(dataset_name, "all")

/data/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''可以直接print 看一下数据集的概况'''
print(raw_dataset)

'''由于返回的是train 和 test 两个split 也可以直接像字典一样进行索引'''
print(type(raw_dataset['train']))
print(raw_dataset['train'])

print(raw_dataset['train'][0])

DatasetDict({
    test: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 4934
    })
    train: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 87866
    })
    validation: Dataset({
        features: ['example_id', 'article', 'answer', 'question', 'options'],
        num_rows: 4887
    })
})
<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['example_id', 'article', 'answer', 'question', 'options'],
    num_rows: 87866
})
{'example_id': 'high19088.txt', 'article': 'Last week I talked with some of my students about what they wanted to do after they graduated, and what kind of job prospects  they thought they had.\nGiven that I teach students who are training to be doctors, I was surprised do find that most thought that they would not be able to get the jobs they wanted without "outside help". "What kind of help is that?" I asked, expecting them to tell me th

# 数据预处理 (Tokenization)
接着我们需要加载 模型对应的分词器对数据进行处理

BERT类模型在预训练时，使用的两个特殊token：[CLS] [SEP]，前者是用于提取全局信息的，后者则是用于划分句子的

这里我们需要了解Multiple Choice QA任务，encoder类模型所期待的输入格式：为每个选项构建一个独立的输入序列，并将它们组合起来。
- 格式 (单个选项序列):
  - [CLS] context [SEP] question + option_i [SEP]
  - 当然，也可以分为三部分输入，但是bert类的tokenizer只能处理两个输入，只能自己重写tokenizer

模型输入张量: (batch_size, num_choices, sequence_length)

*步骤*：
以下代码是基于batch 处理的
1. 构建context列表，对每个instance的 article复制4次
2. 构建question + option_i，i=0~3
3. 先对所有输入进行展平，进行tokenize
4. 重塑为 (nums_instances, nums_choices, max_length)返回

In [ ]:
from transformers import RobertaTokenizer

model_checkpoint = "/data/Weights/roberta/roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)


def preprocess(instances):
    context = [[article] * 4 for article in instances["article"]]
    questions = [q for q in instances["question"]]
    
    question_choice = [
        [f"{ques} {instances['options'][i][j]}" for j in range(4)] 
        for i, ques in enumerate(questions)
    ]
    
    context = sum(context, [])                  #len = (len(instances) * 4)
    question_choice = sum(question_choice, [])  

    tokenized_instances = tokenizer(
        context,
        question_choice,
        max_length=512,
        padding=False,            # 这里我们之后使用DataCollator进行batch内的动态填充
        truncation="only_first",  # 优先截断第一个序列(文章)
    )
    '''返回字典
    dict_keys(['input_ids', 'attention_mask'])
    'input_ids' shape:(16, max_length)
    '''
    
    # 创建标签列
    labels = [ord(l)-ord('A') for l in instances['answer']]
    
    # reshape -> (nums_instances, nums_choices, max_length)
    results = {
        k: [v[i:i+4] for i in range(0, len(v), 4)] 
        for k, v in tokenized_instances.items()
    }
    results["labels"] = labels
    
    return results

c = preprocess(raw_dataset["train"][0:4])

print(c)
print(f"shape of input_ids :({len(c['input_ids'])},{len(c['input_ids'][0])},{len(c['input_ids'][0][0])})")


{'input_ids': [[[0, 10285, 186, 38, 3244, 19, 103, 9, 127, 521, 59, 99, 51, 770, 7, 109, 71, 51, 8505, 6, 8, 99, 761, 9, 633, 5108, 1437, 51, 802, 51, 56, 4, 50118, 18377, 14, 38, 6396, 521, 54, 32, 1058, 7, 28, 3333, 6, 38, 21, 3911, 109, 465, 14, 144, 802, 14, 51, 74, 45, 28, 441, 7, 120, 5, 1315, 51, 770, 396, 22, 35301, 244, 845, 22, 2264, 761, 9, 244, 16, 14, 1917, 38, 553, 6, 4804, 106, 7, 1137, 162, 14, 51, 74, 240, 10, 1437, 1437, 50, 284, 1441, 7, 244, 106, 66, 4, 50118, 113, 27526, 21712, 41066, 65, 6849, 4, 50118, 100, 21, 1256, 23438, 30, 14, 1263, 4, 85, 1302, 14, 5, 11295, 9, 452, 32, 3150, 2882, 7, 213, 223, 5, 7023, 7, 120, 789, 9, 643, 77, 24, 606, 7, 562, 10, 633, 479, 50118, 3762, 1816, 174, 162, 14, 79, 21, 2811, 3012, 7, 712, 69, 6958, 4, 22, 1213, 1108, 110, 5856, 6, 342, 11, 780, 9148, 34242, 6, 8, 5764, 3003, 5, 4044, 227, 5, 80, 3587, 9, 5, 9013, 25, 24, 769, 12, 571, 13415, 6, 47, 64, 120, 23, 513, 195, 25434, 26647, 2901, 50118, 3750, 14, 477, 6, 38, 21, 6649

In [4]:
# 进行批处理，这里我们直接在map函数中将文本信息全部删除即可
tokenized_dataset = raw_dataset.map(preprocess, batched=True,remove_columns=raw_dataset["train"].column_names, num_proc=4)

In [5]:
tokenized_dataset['train'][0]

{'input_ids': [[0,
   10285,
   186,
   38,
   3244,
   19,
   103,
   9,
   127,
   521,
   59,
   99,
   51,
   770,
   7,
   109,
   71,
   51,
   8505,
   6,
   8,
   99,
   761,
   9,
   633,
   5108,
   1437,
   51,
   802,
   51,
   56,
   4,
   50118,
   18377,
   14,
   38,
   6396,
   521,
   54,
   32,
   1058,
   7,
   28,
   3333,
   6,
   38,
   21,
   3911,
   109,
   465,
   14,
   144,
   802,
   14,
   51,
   74,
   45,
   28,
   441,
   7,
   120,
   5,
   1315,
   51,
   770,
   396,
   22,
   35301,
   244,
   845,
   22,
   2264,
   761,
   9,
   244,
   16,
   14,
   1917,
   38,
   553,
   6,
   4804,
   106,
   7,
   1137,
   162,
   14,
   51,
   74,
   240,
   10,
   1437,
   1437,
   50,
   284,
   1441,
   7,
   244,
   106,
   66,
   4,
   50118,
   113,
   27526,
   21712,
   41066,
   65,
   6849,
   4,
   50118,
   100,
   21,
   1256,
   23438,
   30,
   14,
   1263,
   4,
   85,
   1302,
   14,
   5,
   11295,
   9,
   452,
   32,
   3150,
   2882,
  

In [6]:
# 设置数据集格式为 PyTorch tensors (如果使用 TensorFlow 则设置为 "tf")
tokenized_dataset.set_format("torch")

这里稍微介绍一下DataCollator进行padding和在tokenizer进行padding的区别：
使用 DataCollator 进行**动态填充（Dynamic Padding）**通常比在 Tokenizer 阶段进行**全局填充（Global Padding）**更高效、更灵活。其会根据**batch内的最长长度进行填充**，而不是全局的max_length

但是因为DataCollatorWithPadding是针对一个一维序列数据，即 (bs, seq_len)，而我们的数据是二维序列 -> (bs, choices, seq_len)，所以需要自定义一个DataCollator

最后会介绍一下其他常用的DataCollator

In [7]:
import torch
from transformers import DataCollatorWithPadding

# 选择DataCollatorWithPadding作为父类，修改其call的实现
class MultipleChoiceDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # 展开所有选项到批次维度
        flattened_features = [
            {k: v[i] for k, v in feature.items() if k != "labels"}
            for feature in features
            for i in range(len(feature["input_ids"])) 
        ]
        
        # 使用父类call进行一维序列的填充
        batch = super().__call__(flattened_features)
        
        # 恢复选项维度 (batch_size, num_choices, seq_len)
        batch = {k: v.view(len(features), -1, v.size(-1)) for k, v in batch.items()}
        
        # 添加labels
        if "labels" in features[0]:
            batch["labels"] = torch.tensor([f["labels"] for f in features])
            
        return batch

data_collator = MultipleChoiceDataCollator(
    tokenizer=tokenizer,
    padding="longest",
    pad_to_multiple_of=8,
    return_tensors="pt"
)

In [8]:
# 传入四个数据进行查看
batch = data_collator([tokenized_dataset['train'][i] for i in range(4)])
batch

{'input_ids': tensor([[[    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1]],
 
         [[    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1]],
 
         [[    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1]],
 
         [[    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1],
          [    0, 10285,   186,  ...,     1,     1,     1]]]),
 'attention_mas

#  加载RoBERTa预训练模型
加载带有适合下游任务头部的模型。对于文本分类，我们使用 AutoModelForSequenceClassification。其就是在最后加了层MLP

In [9]:
from transformers import RobertaForMultipleChoice, TrainingArguments, Trainer
import numpy as np
import evaluate

model = RobertaForMultipleChoice.from_pretrained(model_checkpoint)
print(model)


Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at /data/Weights/roberta/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [10]:
# 可以看看输出
import torch

dummy_input = tokenized_dataset['train'][0]
for k, v in dummy_input.items():
    dummy_input[k] = v.unsqueeze(0)

with torch.no_grad():
    outputs = model(**dummy_input)

print(outputs)

MultipleChoiceModelOutput(loss=tensor(1.3860), logits=tensor([[0.1880, 0.1879, 0.1881, 0.1872]]), hidden_states=None, attentions=None)


#  定义评估指标
选择适合任务的评估指标。对于多项选择的QA任务，可以看做一个四分类问题，使用accuracy即可

In [11]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": (predictions == eval_pred.label_ids).mean()}


# RoBERTa模型训练

In [12]:
training_args = TrainingArguments(
    output_dir="./results/roberta-epoch5",              # 输出目录，保存模型和日志
    eval_strategy="epoch",         # 每个 epoch 结束后进行评估
    save_strategy="epoch",               # 每个 epoch 结束后保存模型
    learning_rate=2e-5,                  # 学习率
    per_device_train_batch_size=16,      # 训练批次大小
    per_device_eval_batch_size=16,       # 评估批次大小
    num_train_epochs=5,                  # 训练轮数
    weight_decay=0.01,                   # 权重衰减
    load_best_model_at_end=True,         # 训练结束后加载最佳模型
    metric_for_best_model="accuracy",    # 以 accuracy 指标判断最佳模型 (需与 compute_metrics 返回的 key 匹配)
    report_to="tensorboard",             # 可以选择 tensorboard, wandb 等
    save_total_limit=2,
    label_names=["labels"]
)

这里吸取之前的教训，我们不进行全量微调，采用LORA微调，这里要注意使用get_peft_model后会冻结其他层，包括classifier，所以需要手动解冻一下

In [13]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

peft_config = LoraConfig(
        r=16,                        # LoRA 的秩 (Rank)
        lora_alpha=32,               # LoRA 缩放因子
        lora_dropout=0.1,            # LoRA 层的 Dropout
        target_modules=["query", "value"], 
        bias="none",                
    )

model = get_peft_model(model, peft_config)

# 记得解冻classifier 和 pooler的梯度计算
for name, param in model.named_parameters():
    if 'classifier' in name or 'pooler' in name:
        param.requires_grad = True

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

[2025-05-07 14:34:59,620] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/anaconda3/envs/llm/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status


In [15]:
# 查看一下初始精度，并验证能否正常运行
evalres = trainer.evaluate()
print(evalres)

/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 1.3862849473953247, 'eval_accuracy': 0.2596685082872928, 'eval_runtime': 51.3462, 'eval_samples_per_second': 95.178, 'eval_steps_per_second': 2.98}


In [16]:
# 4090 * 2 训练约3h
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.386800,1.352846,0.403520
2,1.141900,1.051512,0.562922
3,1.085700,0.998876,0.588909
4,1.055000,0.973867,0.602005
5,1.050700,0.967764,0.605689


/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/data/anaconda3/envs/llm/lib/python3.10/site

TrainOutput(global_step=13730, training_loss=1.1581273611670242, metrics={'train_runtime': 9983.9119, 'train_samples_per_second': 44.004, 'train_steps_per_second': 1.375, 'total_flos': 4.6460041954632e+17, 'train_loss': 1.1581273611670242, 'epoch': 5.0})

# 模型推理

这里使用hf 的pipeline来进行模型的推理。对于使用标准 hf 训练得到的模型权重，直接传入目录即可。如果是经过其他包装的模型需要重新定义模型

In [62]:
model = RobertaForMultipleChoice.from_pretrained("/data/Weights/roberta/roberta-base")
model = PeftModel.from_pretrained(model, "./results/roberta-epoch10/checkpoint-27460")
print(model)


Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at /data/Weights/roberta/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModel(
  (base_model): LoraModel(
    (model): RobertaForMultipleChoice(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (d

merge_and_unload() 是对部分支持权重合并的peft模型，将微调的权重整合到原模型，并移除相关的peft设置

支持的peft方式有：
- LORA
- QLORA
- $IA^{3}$


In [23]:
model= model.merge_and_unload()
print(model)

RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

由于没有multiple-choices的pipeline，所以这里我们随便学习一下如何自定义pipeline

使用 `Pipeline` 类创建一个问答多项选择任务的 pipeline 时，它的**主要输入**是一个或多个包含**问题 (question)** ，**上下文 (context)** 和 **选项（options）** 的字典。

具体来说，输入形式：

1.  **多个问答对（用于批量处理）：**
    一个列表，列表中的每个元素都是一个如上所示的字典。
    ```python
    [
        {
            'question': '第一个问题',
            'article': '第一个上下文',
            'options': ['opt1','opt2','opt3','opt4']
        },
        {
            'question': '第二个问题',
            'article': '第二个上下文',
            'options': ['opt1','opt2','opt3','opt4']
        },
        # ... 
    ]
    ```


### `Pipeline`的参数


| 参数名称          | 类型                                                                 | 默认值                                                              | 描述                                                                                                                                                                                             |
| :---------------- | :------------------------------------------------------------------- | :------------------------------------------------------------------ | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `model`           | `PreTrainedModel` or `TFPreTrainedModel`                             | 无                                                                  | 用于进行预测的模型。必须继承自 PyTorch 的 `PreTrainedModel` 或 TensorFlow 的 `TFPreTrainedModel`。                                                                                                |
| `tokenizer`       | `PreTrainedTokenizer`                                                | 无                                                                  | 用于对模型输入数据进行编码的 Tokenizer。必须继承自 `PreTrainedTokenizer`。                                                                                                                        |
| `feature_extractor` | `SequenceFeatureExtractor` (可选)                                    | 无                                                                  | 用于对模型输入数据进行编码的 Feature Extractor。必须继承自 `SequenceFeatureExtractor`。                                                                                                            |
| `image_processor` | `BaseImageProcessor` (可选)                                          | 无                                                                  | 用于对模型输入图像数据进行编码的 Image Processor。必须继承自 `BaseImageProcessor`。                                                                                                                |
| `processor`       | `ProcessorMixin` (可选)                                              | 无                                                                  | 用于对模型输入数据进行编码的 Processor。必须继承自 `ProcessorMixin`。Processor 是一个复合对象，可能包含 `tokenizer`, `feature_extractor` 和 `image_processor`。                                    |
| `modelcard`       | `str` or `ModelCard` (可选)                                          | 可选 (无特定默认值对象)                                               | 与此 Pipeline 相关的模型卡。                                                                                                                                                                     |
| `framework`       | `str` (可选)                                                         | 自动检测（基于安装的框架，或模型的框架，或默认为 "pt"）                | 要使用的框架，可以是 "pt" (PyTorch) 或 "tf" (TensorFlow)。指定的框架必须已安装。如果未指定，将根据当前安装的框架自动确定。如果安装了两个框架且未指定模型，则默认为 PyTorch。                           |
| `task`            | `str` (可选)                                                         | `""`                                                                | Pipeline 的任务标识符。                                                                                                                                                                          |
| `num_workers`     | `int` (可选)                                                         | `8`                                                                 | 当 Pipeline 使用 DataLoader 时 (传递数据集时，或 PyTorch 模型在 GPU 上运行时)，要使用的 worker 数量。                                                                                              |
| `batch_size`      | `int` (可选)                                                         | `1`                                                                 | 当 Pipeline 使用 DataLoader 时 (传递数据集时，或 PyTorch 模型在 GPU 上运行时)，要使用的批次大小。请阅读关于 Pipeline 批处理的文档了解更多信息。                                                    |
| `args_parser`     | `ArgumentHandler` (可选)                                             | 无                                                                  | 负责解析提供的 Pipeline 参数的对象引用。                                                                                                                                                           |
| `device`          | `int` or `torch.device` or `str` (可选)                              | `-1`                                                                | CPU/GPU 支持的设备序号。设置为 -1 使用 CPU，正数使用对应的 CUDA 设备 ID。您也可以传递原生的 `torch.device` 对象或字符串。                                                                            |
| `torch_dtype`     | `str` or `torch.dtype` (可选)                                        | 无                                                                  | 直接作为 `model_kwargs` 传递 (只是一个更简单的快捷方式)，用于指定模型使用的精度 (例如 `torch.float16`, `torch.bfloat16` 或 `"auto"`)。                                                            |
| `binary_output`   | `bool` (可选)                                                        | `False`                                                             | 标志，指示 Pipeline 的输出应该是序列化格式 (例如 pickle) 还是原始输出数据 (例如文本)。                                                                                                            |

**总结：**

*   `model`, `tokenizer`, `feature_extractor`, `image_processor`, `processor` 是 Pipeline 的核心组件，您通常需要至少提供 `model` 和一个或多个处理输入数据的组件 (`tokenizer` 用于文本，`feature_extractor`/`image_processor` 用于图像/音频等，或使用复合的 `processor`)。

### **创建自定义 Pipeline 的核心步骤：**

自定义 Pipeline 通常涉及到继承 Hugging Face Transformers 库中的 `Pipeline` 基类，并重写其核心方法。以下是关键步骤和主要方法：

1.  **重写核心方法**:
    * **`_init_(model, tokenizer, **kwargs)`**:
        * **作用**: 没有啥特殊的初始化要求也可以不写，如果写了记得要调用父类的 `__init__` 方法，并加载你的模型和分词器 (或其他必要的组件)。

    * **`_sanitize_parameters(**kwargs)`**:
        * **作用**: 这个方法在 `preprocess` 之前被调用。它的主要职责是验证和清理传递给 Pipeline 的参数 (`__call__` 方法的参数)。
        * **返回**: 一个包含预处理、前向传播和后处理所需参数的字典。
        * **示例**:
            ```python
            def _sanitize_parameters(self, **kwargs):
                preprocess_kwargs = {}
                if "my_custom_arg" in kwargs:
                    preprocess_kwargs["my_custom_arg"] = kwargs["my_custom_arg"]
                return preprocess_kwargs, {}, {} # (preprocess_kwargs, forward_kwargs, postprocess_kwargs)
            ```

    * **`preprocess(self, inputs, **preprocess_kwargs)`**:
        * **作用**: 这个方法接收原始输入 (例如文本、图像路径等)，并将其转换为模型可以理解的格式 (通常是 PyTorch 或 TensorFlow 张量)。
        * **`inputs`**: Pipeline 调用时传入的原始数据。
        * **`**preprocess_kwargs`**: 从 `_sanitize_parameters` 返回的预处理参数。
        * **返回**: 一个包含模型输入的字典，键通常是模型期望的输入名称 (如 `input_ids`, `attention_mask`)。
        * **示例 (文本分类)**:
            ```python
            def preprocess(self, text_input, **preprocess_kwargs):
                # 使用分词器处理文本
                return self.tokenizer(text_input, return_tensors=self.framework, truncation=True, padding=True)
            ```

    * **`_forward(self, model_inputs, **forward_kwargs)`**:
        * **作用**: 这个方法接收 `preprocess` 方法的输出，并将其传递给模型进行实际的推理。
        * **`model_inputs`**: `preprocess` 方法返回的模型输入。
        * **`**forward_kwargs`**: 从 `_sanitize_parameters` 返回的前向传播参数。
        * **返回**: 模型的原始输出。
        * **示例**:
            ```python
            def _forward(self, model_inputs, **forward_kwargs):
                # 将输入传递给模型
                return self.model(**model_inputs)
            ```

    * **`postprocess(self, model_outputs, **postprocess_kwargs)`**:
        * **作用**: 这个方法接收模型的原始输出，并将其转换为用户友好的、可理解的格式。
        * **`model_outputs`**: `_forward` 方法返回的模型输出。
        * **`**postprocess_kwargs`**: 从 `_sanitize_parameters` 返回的后处理参数。
        * **返回**: Pipeline 的最终输出。
        * **示例 (文本分类，返回标签和分数)**:
            ```python
            def postprocess(self, model_outputs, **postprocess_kwargs):
                logits = model_outputs.logits
                probabilities = torch.softmax(logits, dim=-1)
                scores = probabilities.tolist()[0] # 假设批处理大小为1
                # 假设你有 id 到标签的映射 self.model.config.id2label
                results = []
                for i, score in enumerate(scores):
                    results.append({"label": self.model.config.id2label[i], "score": score})
                return sorted(results, key=lambda x: x["score"], reverse=True) # 按分数排序
            ```

In [ ]:
from transformers import Pipeline
device = "cuda:0"

# 简单化，默认都是4个选项

class MultipleChoicesPipeline(Pipeline):
    def __init__(self, model, tokenizer = None, feature_extractor = None, image_processor = None, processor = None, modelcard = None, framework = None, task = "", args_parser = None, device = None, torch_dtype = None, binary_output = False, **kwargs):
        self.labels = None
        self.options = None
        super().__init__(model, tokenizer, feature_extractor, image_processor, processor, modelcard, framework, task, args_parser, device, torch_dtype, binary_output, **kwargs)
    
    def _sanitize_parameters(self, **kwargs):
        """
        我们没有自定义参数，直接返回空字典。
        """
        return {}, {}, {}
    
    # 参照之前数据集处理的写法即可
    def preprocess(self, input_, **preprocess_parameters):
        
        if isinstance(input_, dict):
            input_ = [input_]
            
        article = list(map(lambda x:x["article"], input_))
        question = list(map(lambda x:x["question"], input_))
        options = list(map(lambda x:x["options"], input_))
        answer = list(map(lambda x:x["answer"], input_))
        
        self.labels = [ord(l)-ord('A') for l in answer]
        self.options = options
        
        context = [[article_] * 4 for article_ in article]
        question_choice = [
                [f"{ques} {options[i][j]}" for j in range(4)] 
                for i, ques in enumerate(question)
            ]
        
        
        context = sum(context, [])                  #len = (len(instances) * 4)
        question_choice = sum(question_choice, []) 
        
        tokenized_instances = tokenizer(
                                    context,
                                    question_choice,
                                    max_length=512,
                                    padding=True,            
                                    truncation="only_first",  # 优先截断第一个序列(文章)
                                    return_tensors="pt",
                                    )
        
        # 因为后续没有datacollator，所以这里要将输出都变为tensor
        results = {
                k: torch.stack([v[i:i+4] for i in range(0, len(v), 4)])
                for k, v in tokenized_instances.items()
                }
        return results
    
    def _forward(self, model_inputs, **forward_kwargs):
                return self.model(**model_inputs)
    
    def postprocess(self, model_outputs, **postprocess_parameters):
        
        logits = model_outputs.logits
        scores = torch.softmax(logits, dim=-1)
        
        batch_size = logits.shape[0]
        results = []
        for i in range(batch_size):
            question_scores = scores[i] # Shape (num_options,)

            max_score = torch.max(question_scores).item()
            predicted_index = torch.argmax(question_scores).item()
            label = self.labels[i]

            original_options = self.options[i]
            predicted_option_text = original_options[predicted_index] 

            result = {
                "predicted_option_index": predicted_index,
                "label": label,
                "predicted_option_text": predicted_option_text,
                "score": max_score,
            }

            results.append(result)
        return results



In [65]:
pipeline = MultipleChoicesPipeline(model, tokenizer)

input_ = [raw_dataset["test"][i] for i in range(4)]
res = pipeline(input_)
res


Device set to use cuda:0


[[{'predicted_option_index': 0,
   'label': 2,
   'predicted_option_text': 'Measure the depth of the river',
   'score': 0.29531028866767883}],
 [{'predicted_option_index': 3,
   'label': 3,
   'predicted_option_text': 'Nancy took hold of the rope and climbed into the helicopter.',
   'score': 0.28053829073905945}],
 [{'predicted_option_index': 1,
   'label': 0,
   'predicted_option_text': 'They used helicopters to help carry cows.',
   'score': 0.2699262797832489}],
 [{'predicted_option_index': 0,
   'label': 1,
   'predicted_option_text': 'our values and lifestyles are in no field of human activity',
   'score': 0.28568899631500244}]]

# DataCollator

| **Data Collator 类型**               | **主要用途**                     | **核心功能**                                                                 | **关键参数**                                                                 | **适用任务举例**                                                                 |
|--------------------------------------|----------------------------------|-----------------------------------------------------------------------------|-----------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| **DataCollatorWithPadding**          | 通用序列填充                     | 动态填充 `input_ids`, `attention_mask` 等，使批次内序列长度一致                          | `tokenizer`（必需）, `padding="longest"`, `pad_to_multiple_of`               | 文本分类、抽取式问答、多项选择任务                                                  |
| **DataCollatorForTokenClassification** | 词元级分类任务                   | 填充 `input_ids` 和 `labels`，并用 `-100` 忽略填充位置的损失计算                         | `tokenizer`, `label_pad_token_id=-100`                                      | 命名实体识别 (NER)、词性标注 (POS)                                                 |
| **DataCollatorForSeq2Seq**           | 序列到序列任务                   | 分别填充编码器输入和解码器标签，自动生成 `decoder_input_ids`（需提供 `model`）               | `tokenizer`, `model`（推荐）, `label_pad_token_id=-100`                     | 机器翻译、文本摘要、生成式问答                                                     |
| **DataCollatorForLanguageModeling**  | 语言模型预训练                   | 动态执行掩码语言建模 (MLM)，随机替换 15% 的 token 并生成 `labels`                          | `tokenizer`, `mlm=True`, `mlm_probability=0.15`                             | BERT/RoBERTa 预训练、领域自适应训练                                                 |
| **DataCollatorForWholeWordMask**     | 全词掩码预训练                   | 类似 `DataCollatorForLanguageModeling`，但以整个词为单位进行掩码                             | 同 `DataCollatorForLanguageModeling`                                        | 需要更高质量掩码的预训练任务                                                        |
| **DefaultDataCollator**              | 通用默认处理                     | 仅将数据转换为张量，不进行填充或特殊处理（需提前确保数据形状一致）                               | 无特殊参数                                                                   | 非序列数据（如图像特征）、已手动处理填充的数据                                         |


### **关键注意事项**
1. **动态填充效率**：优先使用 `padding="longest"`（而非 `max_length`），避免不必要的计算浪费。
2. **模型参数传递**：对 Seq2Seq 任务，务必向 `DataCollatorForSeq2Seq` 传递 `model` 以正确处理 `decoder_input_ids`。
3. **标签填充值**：分类任务中 `-100` 是默认的忽略索引（PyTorch 的 `CrossEntropyLoss` 会自动跳过）。
4. **调试技巧**：通过 `print(batch.keys())` 和 `print(batch["input_ids"].shape)` 检查批次格式是否符合预期。

根据任务类型选择匹配的 Collator，能显著减少预处理代码的复杂性并提升训练效率。